In [1]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
import cv2
import numpy

import pickle

2024-06-01 20:52:40.083779: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 20:52:44.332918: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import imageio.v3 as iio
import os
from datetime import timedelta
import sys
from pathlib import Path 

## Подготовка данных из видео

Предполагается, что видео для инференса подгружаются в папку data/test/videos

### Служебные функции

In [3]:
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return (result + ".00").replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

In [4]:
def main(video_name, model_file):
    """
    Функция выделяет в видео отрезки с нарушенями и за __ секунд до нарушения,
    создает кадры и сохраняет их в заданную папку.
    
    Args:
        video_name - название видео
        model_file - имя файла с моделью
    """
    # получить FPS видео
    metadata = iio.immeta(os.path.join(os.getcwd(), 'data', 'test', 'videos', video_name),exclude_applied=False)
    fps = metadata['fps']
    clip_duration = metadata['duration']
    # если SAVING_FRAMES_PER_SECOND выше видео FPS, то установите его на FPS (как максимум)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # получить список длительностей   
    # список тпйм-кодов для стоп-кадров    
    list_time_codes = []    
    for i in np.arange(0, clip_duration, 1 / saving_frames_per_second):
        list_time_codes.append(i)
    # получить список длительностей для сохранения
    # запускаем цикл
    count = 0    
    res = {'frametime': [],
            'score': []}
    model = pickle.load(open(os.path.join(os.getcwd(), model_file), 'rb'))
    
    for frame in iio.imiter(os.path.join(os.getcwd(), 'data', 'test', 'videos', video_name), plugin="pyav"):
        frame_duration = count / fps
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = list_time_codes[0]
        except IndexError:
            # список пуст, все кадры длительности сохранены
            break
        count += 1
        if frame_duration >= closest_duration:
            res['frametime'].append(count / fps)
            if model_file == 'baseline.pkl':
                #print(cv2.resize(frame,(224, 224)).shape)
                res['score'].append(model.predict(cv2.resize(frame,(224, 224)).reshape([1, 224, 224, 3])))
            elif model_file == 'ur_model_filename':
                pass # добавить свою логику
            else:
                print('ПРОПИШИ ЛОГИКУ ПОД СВОЙ ПИКЛ С МОДЕЛЬЮ В СКРИПТЕ')
                break
            try:
                list_time_codes.pop(0)
            except IndexError:
                pass
        
    res = pd.DataFrame(res)
    res['model'] = model_file
    res['filename'] = video_name
    return res

## Инференс

In [5]:
SAVING_FRAMES_PER_SECOND = 1 # кол-во кадров  в секунду
MODEL_FILE = 'baseline.pkl'

In [6]:
save_result_path = os.path.join(os.getcwd(), MODEL_FILE.split('.')[0], 'scores.csv')


inf_result = pd.DataFrame()

for f in tqdm([
x for x in os.listdir(os.path.join(os.getcwd(), 'data', 'test', 'videos')) if x.lower().endswith('.mp4')
]):
    video_file = os.path.join(os.getcwd(), 'data', 'test', 'videos', f'{f}')  
    video_df = main(f, MODEL_FILE)   
    inf_result = pd.concat([inf_result, video_df], ignore_index=True)
inf_result.to_csv(save_result_path)

  0%|          | 0/1 [00:00<?, ?it/s]2024-06-01 20:53:04.687201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31136 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:8c:00.0, compute capability: 7.0
2024-06-01 20:53:32.006368: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8900


1/1 [==============================] - 0s 31ms/step


100%|██████████| 1/1 [00:46<00:00, 46.26s/it]
